# DataJoint U24 - Workflow DeepLabCut

## Workflow Automation

In the previous notebook [03-Process](./03-Process.ipynb), we ran through the workflow in detailed steps. For daily running routines, the current notebook provides a more succinct and automatic approach to run through the pipeline using some utility functions in the workflow.

The commands here run a workflow using [example data](https://downgit.github.io/#/home?url=https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) from the [00-DownloadData](./00-DataDownload_Optional.ipynb) notebook, but note where placeholders could be changed for a different dataset.

In [1]:
import os; from pathlib import Path
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-deeplabcut', ("Please move to the "
                                                              + "workflow directory")
from workflow_deeplabcut.pipeline import lab, subject, session, train, model

Connecting cbroz@dss-db.datajoint.io:3306


If you previously completed the [03-Process notebook](./03-Process.ipynb), you may want to delete the contents ingested there, to avoid duplication errors.

In [ ]:
# (session.Session & 'subject="subject6"').delete()
# train.TrainingParamSet.delete()
# train.VideoSet.delete()

## Ingestion of subjects, sessions, videos and training parameters

Refer to the `user_data` folder in the workflow.

1. Fill subject and session information in files `subjects.csv` and `sessions.csv`
2. Fill in recording and parameter information in `recordings.csv` and `config_params.csv`
    + Add both training and estimation videos to the recording list
    + Additional columns in `config_params.csv` will be treated as model training parameters
3. Run automatic scripts prepared in `workflow_deeplabcut.ingest` for ingestion: 
    + `ingest_subjects` for `subject.Subject`
    + `ingest_sessions` - for session tables `Session`, `SessionDirectory`, and `SessionNote`
    + `ingest_dlc_items` - for ...
        - `train.ModelTrainingParamSet`
        - `train.VideoSet` and the corresponding `File` part table
        - `model.VideoRecording` and the corresponding `File` part table
        - `model.RecordingInfo`

In [3]:
from workflow_deeplabcut.ingest import ingest_subjects, ingest_sessions, ingest_dlc_items
ingest_subjects(); ingest_sessions(); ingest_dlc_items()


---- Inserting 1 entry(s) into subject ----

---- Inserting 3 entry(s) into session ----

---- Inserting 3 entry(s) into session_directory ----

---- Inserting 3 entry(s) into session_note ----

---- Inserting 3 entry(s) into #model_training_param_set ----

---- Inserting 2 entry(s) into video_set ----

---- Inserting 8 entry(s) into video_set__file ----

---- Inserting 2 entry(s) into video_recording ----

---- Inserting 2 entry(s) into video_recording__file ----

---- Inserting 2 entry(s) into _recording_info ----


## Setting project variables

1. Set your root directory in your DataJoint config file, under `custom` as `dlc_root_data_dir`. 

In [4]:
import datajoint as dj; dj.config.load('dj_local_conf.json')
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
                          'openfield-Pranav-2018-10-30')            # DLC project dir
config_path = (data_dir / 'config.yaml')

2. For the purposes of this demo, we'll ask DeepLabCut to structure the demo config file with `load_demo_data`. If you already did this in the [00-DataDownload notebook](./00-DataDownload_Optional.ipynb), skip this step.

In [ ]:
from deeplabcut.create_project.demo_data import load_demo_data
load_demo_data(config_path)

3. For this demo, we generate a copy to show pose estimation. This is `recording_id` 2 in `recordings.csv`. If you already did this in the [00-DataDownload notebook](./00-DataDownload_Optional.ipynb), skip this step.

In [5]:
vid_path = str(data_dir).replace(" ", "\ ") + '/videos/m3v1mp4'
cmd = (f'ffmpeg -n -hide_banner -loglevel error -ss 0 -t 2 -i {vid_path}.mp4 -vcodec copy '
       + f'-acodec copy {vid_path}-copy.mp4') # New video copy, first 2 seconds
os.system(cmd)

File '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4-copy.mp4' already exists. Exiting.


256

4. Pair training files with training parameters, and launch training.

In [ ]:
key={'paramset_idx':1,'training_id':1,'video_set_id':1, 
     'project_path':'openfield-Pranav-2018-10-30/'}
train.TrainingTask.insert1(key, skip_duplicates=True)
train.ModelTraining.populate()

5. Add this model to the `Model` table and evaluate.
   - Include a user-friendly `model_name`
   - Include the relative path for the project's `config.yaml`
   - Add `shuffle` and `trainingsetindex`
   - `insert_new_model` will prompt before inserting, but this can be skipped with `prompt=False`

In [ ]:
model.Model.insert_new_model(model_name='OpenField-5',dlc_config=config_path,
                             shuffle=1,trainingsetindex=0, paramset_idx=1, 
                             prompt=True, # True is the default behavior
                             model_description='Open field model trained 5 iterations')
model.ModelEvaluation.populate()

6. Auto-populate the `RecordingInfo` table with relevant fields.

In [7]:
model.RecordingInfo.populate()
model.RecordingInfo()

subject,session_datetime,recording_id,px_height height in pixels,px_width width in pixels,nframes number of frames,fps (Hz) frames per second,recording_datetime Datetime for the start of the recording,recording_duration video duration in seconds
subject5,2020-04-15 11:16:38,3,747,832,60,30,None,2.0
subject6,2021-06-03 14:43:10,2,480,640,63,30,None,2.1


7. Add a pose estimation task, and launch pose estimation.
   - Get all primary key information for a given recording
   - Add the model and `task_mode` (i.e., load vs. trigger) to be applied
   - Add any additional analysis parameters for `deeplabcut.analyze_videos`

In [ ]:
key=(model.VideoRecording & 'recording_id=2').fetch1('KEY')
key.update({'model_name': 'OpenField-5', 'task_mode': 'trigger'})
analyze_params={'save_as_csv':True} # add any others from deeplabcut.analyze_videos
model.PoseEstimationTask.insert_estimation_task(key,params=analyze_params)
model.PoseEstimation.populate()

8. Retrieve estimated position data.

In [8]:
model.PoseEstimation.get_trajectory(key)

scorer    OpenField-5                                                       \
bodyparts     leftear                              rightear                  
coords              x           y    z likelihood         x         y    z   
0            5.966216   -4.787393  0.0   0.032592 -1.522350  8.632778  0.0   
1            4.879360   -3.865869  0.0   0.040094 -1.018066  9.007607  0.0   
2            7.582597  141.982101  0.0   0.023750 -1.379965  6.661551  0.0   
3            6.314935   -4.641110  0.0   0.026313 -1.371757  7.696576  0.0   
4            5.441626   -5.161995  0.0   0.030603 -1.186847  7.544275  0.0   
..                ...         ...  ...        ...       ...       ...  ...   
58           2.117847   -3.214342  0.0   0.033657 -1.280500  9.483593  0.0   
59           3.116104   -4.693888  0.0   0.024087  0.219144  8.388535  0.0   
60           3.547711   -4.983343  0.0   0.023138 -0.428721  7.312115  0.0   
61           2.383031   -3.214643  0.0   0.030314 -1.092042  8.263630  0.0   
62           3.069550   -5.201364  0.0   0.022396 -0.602118  6.999699  0.0   

scorer                                                                         \
bodyparts                snout                             tailbase             
coords    likelihood         x          y    z likelihood         x         y   
0           0.053609  2.076265  16.415096  0.0   0.139537  3.148022 -6.657187   
1           0.069977  1.774640  17.406301  0.0   0.176452  2.704738 -8.274201   
2           0.060571  2.861758  13.935984  0.0   0.148534  2.233558 -6.290349   
3           0.059226  2.387363  16.957266  0.0   0.144777  2.781236 -7.757162   
4           0.063246  2.337324  18.026840  0.0   0.148035  2.534595 -9.052482   
..               ...       ...        ...  ...        ...       ...       ...   
58          0.046280  2.093328  17.112661  0.0   0.158959  2.268270 -8.589610   
59          0.053968  3.630704  16.375135  0.0   0.108622  1.831684 -6.993159   
60          0.042809  3.867933  16.821577  0.0   0.097812  1.444253 -7.488105   
61          0.054284  2.453674  17.717480  0.0   0.136344  1.779472 -8.309467   
62          0.049786  4.244837  15.407806  0.0   0.109382  2.311671 -6.654770   

scorer                     
bodyparts                  
coords       z likelihood  
0          0.0   0.054340  
1          0.0   0.067940  
2          0.0   0.044671  
3          0.0   0.058866  
4          0.0   0.055323  
..         ...        ...  
58         0.0   0.051529  
59         0.0   0.040950  
60         0.0   0.031486  
61         0.0   0.046111  
62         0.0   0.037103  

[63 rows x 16 columns]

## Summary and next step

+ This notebook runs through the workflow in an automatic manner.

+ The next notebook [06-Drop](06-Drop_Optional.ipynb) shows how to drop schemas and tables if needed.